In [1]:
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-03-06 15:59:06.157643: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 15:59:06.161028: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-06 15:59:06.197977: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 15:59:06.198016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 15:59:06.199283: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
from crops_package.data import split_data

In [6]:
data = pd.read_csv(f"{os.environ.get('DATA_PATH')}/data.csv")
data['filename'] = os.environ.get("RAW_DATA_PATH") + "/" + data['filename']
data['class'] = data['class'].astype(str)

In [7]:
train_df, val_df, test_df = split_data(data)

In [5]:
len(train_df)

181

In [6]:
from crops_package.data import preprocessor_df
train_generator = preprocessor_df(train_df)
val_generator = preprocessor_df(val_df)

Found 181 validated image filenames belonging to 22 classes.
Found 32 validated image filenames belonging to 15 classes.


# Model initialization

In [11]:
from tensorflow.keras import layers
from tensorflow.keras import models

def initialize_model():
    model = models.Sequential()

    ### First Convolution & MaxPooling
    model.add(layers.Conv2D(32, (4,4), input_shape=(224, 224, 3), activation='relu', padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Second Convolution & MaxPooling
    # model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same'))
    # model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    model.add(layers.Dense(2, activation='relu'))

    ### Last layer - Classification Layer with 10 outputs corresponding to 10 digits
    model.add(layers.Dense(22, activation='softmax'))

    ### Model compilation
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

# fit model with earlystopping

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

model_small = initialize_model()

es = EarlyStopping(patience = 5, verbose = 2)

history_small = model_small.fit(train_generator,
                    validation_data = val_generator,
                    callbacks = [es],
                    epochs = 100,
                    batch_size = 64)

Epoch 1/100
6/6 [==============================] - ETA: 0s - loss: 3.7537 - accuracy: 0.0552

InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/home/mahaut/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/mahaut/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/mahaut/.pyenv/versions/3.10.6/envs/croply/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/mahaut/.pyenv/versions/3.10.6/envs/croply/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/mahaut/.pyenv/versions/3.10.6/envs/croply/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 724, in start

  File "/home/mahaut/.pyenv/versions/3.10.6/envs/croply/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/mahaut/.pyenv/versions/3.10.6/lib/python3.10/asyncio/base_events.py", line 600, in ru

In [24]:
model_small.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 224, 224, 32)      1568      
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 flatten_6 (Flatten)         (None, 401408)            0         
                                                                 
 dense_12 (Dense)            (None, 2)                 802818    
                                                                 
 dense_13 (Dense)            (None, 22)                66        
                                                                 
Total params: 804452 (3.07 MB)
Trainable params: 804452 (3.07 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

## Plotting History

In [27]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label = 'train' + exp_name)
    ax1.plot(history.history['val_loss'], label = 'val' + exp_name)
    ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [28]:
plot_history(history_small)
plt.show()

NameError: name 'history_small' is not defined

In [ ]:
res = model_small.evaluate(X_test_small, y_test_small, verbose = 0)
res